In [88]:
T = 1
sigma = 0.2
r = 0.06
s0 = 100
epsilon = 0.01 #change between 0.01, 0.02, and 0.5
s_bumped = s0 + epsilon
K = 99
N = 365
dt = T/N
M = 10**4

In [108]:
import numpy as np

#stock price paths
np.random.seed(3) #same/different seed for approx_delta for experiment

def pricing(s0,r,T,sigma):
    Z = np.random.normal(0,1)
    S_T = s0*np.exp((r-0.5*sigma**2)*T+sigma*np.sqrt(T)*Z)
    return S_T

print("random price of stock = ", pricing(s0,r,T,sigma))


#option value formula, use twice for bumped and unbumped values

def option_value(s0,r,T,sigma,K,M):
   
    thesum = 0
    
    for i in range(M):
        thesum += max(pricing(s0,r,T,sigma)-K, 0)
        
    average = thesum/M
    price = np.exp(-r*T)*average
    return(price)


#euler approx of delta at time = t:
def approx_delta():
    unbumped = option_value(s0,r,T,sigma,K,M)
    bumped = option_value(s_bumped,r,T,sigma,K,M)
    delta = ((bumped - unbumped) / epsilon)
    return unbumped, bumped, delta

print(approx_delta())

random price of stock =  148.84356880859778
(11.140825538098973, 11.543525256914798, 40.26997188158248)


In [163]:
from scipy.stats import norm
import numpy as np

np.random.seed(3)
random = [np.random.normal(0,1) for x in range(M)]

def MC_ultimate_function():
    
    stocks = []
    stocks_b = []
    
    payoffs = []
    payoffs_b = []
    
    for Z in random:
        S_T = s0*np.exp((r-0.5*sigma**2)*T+sigma*np.sqrt(T)*Z)     
        S_T_bumped = s_bumped*np.exp((r-0.5*sigma**2)*T+sigma*np.sqrt(T)*Z)
    
        stocks.append(S_T)
        stocks_b.append(S_T_bumped)
    
    
        payoff = max(K - S_T, 0)
        payoff_b = max(K - S_T_bumped, 0)
        
        payoffs.append(payoff)
        payoffs_b.append(payoff_b)
        
    average = sum(payoffs) / len(payoffs)
    average_b = sum(payoffs_b) / len(payoffs_b)
    
    price = np.exp(-r*T)*average
    price_b = np.exp(-r*T)*average_b
    
    
    delta = (price_b - price) / epsilon
    
    d1 = np.log(s0/K) + T*(r+(sigma**2/2))/sigma*np.sqrt(T)
    
    BS_delta = norm.cdf(d1) - 1
    
    error = delta - BS_delta
    
    return price, price_b, delta, BS_delta, error


print("bumped: " , MC_ultimate_function()[0])
print("unbumped: " , MC_ultimate_function()[1])
print("delta: " , MC_ultimate_function()[2])
print("BS delta: ", MC_ultimate_function()[3])
print("Difference w/ analytic BS: ", MC_ultimate_function()[4])

bumped:  4.944006317963819
unbumped:  4.940617688104622
delta:  -0.33886298591969677
BS delta:  -0.34088451169481104
Difference w/ analytic BS:  0.0020215257751142746


# digital option

In [91]:
import matplotlib.pyplot as plt

def digital_payoff():
    if pricing(s0,r,T,sigma) < K:
        payoff = 0
    else:
        payoff = 1
    return payoff

def digital_option_value(s0,r,T,sigma,K,M):
   
    thesum = 0
    
    for i in range(M):
        thesum += digital_payoff()
        
    average = thesum/M
    price = np.exp(-r*T)*average
    return(price)

print(digital_option_value(s0,r,T,sigma,K,M))

0.5651528966039077
